In [60]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import string
import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]
doc_complete

['Sugar is bad to consume. My sister likes to have sugar, but not my father.',
 'My father spends a lot of time driving my sister around to dance practice.',
 'Doctors suggest that driving may cause increased stress and blood pressure.',
 'Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better.',
 'Health experts say that Sugar is not good for your lifestyle.']

In [4]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [7]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

doc_clean = [clean(doc).split() for doc in doc_complete]
doc_clean

[['sugar', 'bad', 'consume', 'sister', 'like', 'sugar', 'father'],
 ['father',
  'spends',
  'lot',
  'time',
  'driving',
  'sister',
  'around',
  'dance',
  'practice'],
 ['doctor',
  'suggest',
  'driving',
  'may',
  'cause',
  'increased',
  'stress',
  'blood',
  'pressure'],
 ['sometimes',
  'feel',
  'pressure',
  'perform',
  'well',
  'school',
  'father',
  'never',
  'seems',
  'drive',
  'sister',
  'better'],
 ['health', 'expert', 'say', 'sugar', 'good', 'lifestyle']]

## Create Document-Term Matrix
It is a matrix representing bag of words model. Each document is composed of many terms. So, frequency of those terms/words is represented in this matrix.  
```python
   W1  W2  W3 ...
D1 1   99  
D2
D3
```

In [37]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2)],
 [(2, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(8, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1)],
 [(2, 1),
  (4, 1),
  (18, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)],
 [(5, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]]

In [42]:
# get id for each term
[[(dictionary[id], freq) for id, freq in cp] for cp in doc_term_matrix[:5]]

[[('bad', 1),
  ('consume', 1),
  ('father', 1),
  ('like', 1),
  ('sister', 1),
  ('sugar', 2)],
 [('father', 1),
  ('sister', 1),
  ('around', 1),
  ('dance', 1),
  ('driving', 1),
  ('lot', 1),
  ('practice', 1),
  ('spends', 1),
  ('time', 1)],
 [('driving', 1),
  ('blood', 1),
  ('cause', 1),
  ('doctor', 1),
  ('increased', 1),
  ('may', 1),
  ('pressure', 1),
  ('stress', 1),
  ('suggest', 1)],
 [('father', 1),
  ('sister', 1),
  ('pressure', 1),
  ('better', 1),
  ('drive', 1),
  ('feel', 1),
  ('never', 1),
  ('perform', 1),
  ('school', 1),
  ('seems', 1),
  ('sometimes', 1),
  ('well', 1)],
 [('sugar', 1),
  ('expert', 1),
  ('good', 1),
  ('health', 1),
  ('lifestyle', 1),
  ('say', 1)]]

## LDA (Latent Drichlet Allocation)
LDA uses document-term matrix to model the topic. It breaks down this matrix into 2 matrices, **M1** and **M2** where:  
M1 = document-topic matrix  
M2 = topic-terms matrix

In [43]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel


lda_model = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [51]:
#lda_model.print_topics(num_topics=3, num_words=5)
pprint.pprint(lda_model.print_topics(num_topic))
doc_lda = lda_model[doc_term_matrix]
doc_lda

[(0,
  '0.079*"driving" + 0.045*"suggest" + 0.045*"stress" + 0.045*"blood" + '
  '0.045*"may" + 0.045*"doctor" + 0.045*"increased" + 0.045*"time" + '
  '0.045*"cause" + 0.045*"around"'),
 (1,
  '0.091*"sugar" + 0.064*"sister" + 0.064*"father" + 0.036*"health" + '
  '0.036*"bad" + 0.036*"consume" + 0.036*"expert" + 0.036*"lifestyle" + '
  '0.036*"good" + 0.036*"like"'),
 (2,
  '0.029*"father" + 0.029*"pressure" + 0.029*"drive" + 0.029*"sometimes" + '
  '0.029*"school" + 0.029*"perform" + 0.029*"well" + 0.029*"never" + '
  '0.029*"better" + 0.029*"sister"')]


In [64]:
# Compute Perplexity
# a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix))  #

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.11920134341994

Coherence Score:  0.31970581848144614


In [65]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

/home/paradox/anaconda3/envs/machine-learning/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
/home/paradox/anaconda3/envs/machine-learning/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1     -0.089432  0.0       1        1  57.867287
0      0.089407  0.0       2        1  41.683392
2      0.000026  0.0       3        1   0.449315, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
5     Default  2.000000      sugar  2.000000  30.0000  30.0000
8     Default  1.000000    driving  1.000000  29.0000  29.0000
22    Default  1.000000      drive  1.000000  28.0000  28.0000
28    Default  1.000000  sometimes  1.000000  27.0000  27.0000
26    Default  1.000000     school  1.000000  26.0000  26.0000
25    Default  1.000000    perform  1.000000  25.0000  25.0000
29    Default  1.000000       well  1.000000  24.0000  24.0000
24    Default  1.000000      never  1.000000  23.0000  23.0000
21    Default  1.000000     better  1.000000  22.0000  22.0000
27    Default  1.000000      seems  1.000000  21.0000  21.0000
23    Default  1.000000       feel  1.000000  20.0000  20.0000
3     Default  1.000000       like  1.000000  19.0000  19.0000
34    Default  1.000000        say  1.000000  18.0000  18.0000
0     Default  1.000000        bad  1.000000  17.0000  17.0000
1     Default  1.000000    consume  1.000000  16.0000  16.0000
33    Default  1.000000  lifestyle  1.000000  15.0000  15.0000
31    Default  1.000000       good  1.000000  14.0000  14.0000
30    Default  1.000000     expert  1.000000  13.0000  13.0000
32    Default  1.000000     health  1.000000  12.0000  12.0000
10    Default  1.000000   practice  1.000000  11.0000  11.0000
11    Default  1.000000     spends  1.000000  10.0000  10.0000
9     Default  1.000000        lot  1.000000   9.0000   9.0000
7     Default  1.000000      dance  1.000000   8.0000   8.0000
6     Default  1.000000     around  1.000000   7.0000   7.0000
14    Default  1.000000      cause  1.000000   6.0000   6.0000
12    Default  1.000000       time  1.000000   5.0000   5.0000
16    Default  1.000000  increased  1.000000   4.0000   4.0000
13    Default  1.000000      blood  1.000000   3.0000   3.0000
19    Default  1.000000     stress  1.000000   2.0000   2.0000
20    Default  1.000000    suggest  1.000000   1.0000   1.0000
...       ...       ...        ...       ...      ...      ...
14     Topic3  0.005504      cause  1.037556   0.1660  -3.5583
12     Topic3  0.005504       time  1.037570   0.1660  -3.5584
16     Topic3  0.005503  increased  1.037591   0.1658  -3.5585
13     Topic3  0.005503      blood  1.037599   0.1658  -3.5585
19     Topic3  0.005503     stress  1.037602   0.1658  -3.5585
15     Topic3  0.005503     doctor  1.037606   0.1658  -3.5585
20     Topic3  0.005503    suggest  1.037604   0.1658  -3.5585
17     Topic3  0.005503        may  1.037608   0.1657  -3.5585
22     Topic3  0.005557      drive  1.110377   0.1077  -3.5488
28     Topic3  0.005556  sometimes  1.110394   0.1076  -3.5488
26     Topic3  0.005556     school  1.110403   0.1076  -3.5489
25     Topic3  0.005553    perform  1.110508   0.1070  -3.5493
29     Topic3  0.005552       well  1.110563   0.1067  -3.5496
24     Topic3  0.005550      never  1.110623   0.1064  -3.5499
21     Topic3  0.005548     better  1.110708   0.1060  -3.5502
27     Topic3  0.005543      seems  1.110935   0.1047  -3.5513
23     Topic3  0.005541       feel  1.111024   0.1043  -3.5517
3      Topic3  0.005504       like  1.112497   0.0962  -3.5583
34     Topic3  0.005503        say  1.112508   0.0962  -3.5584
0      Topic3  0.005503        bad  1.112539   0.0961  -3.5585
31     Topic3  0.005503       good  1.112535   0.0960  -3.5585
33     Topic3  0.005503  lifestyle  1.112543   0.0960  -3.5585
1      Topic3  0.005503    consume  1.112547   0.0960  -3.5586
30     Topic3  0.005502     expert  1.112557   0.0959  -3.5586
32     Topic3  0.005502     health  1.112572   0.0959  -3.5587
18     Topic3  0.005559   pressure  1.714639  -0.3263  -3.